# 🚀 İnteraktif JetX Model Eğitmeni (v4.6 - İndirme Özellikli)

### 🛠️ 1. Kurulum (Tek Adım)

**Lütfen sadece bu hücreyi çalıştırın ve tamamlanmasını bekleyin.**

Bu hücre, interaktif arayüzün (butonlar, seçim kutuları) çalışması için gerekli tüm adımları otomatik olarak gerçekleştirir:
1.  **İnteraktif Arayüzü Etkinleştirir:** Colab'ın widget'ları doğru göstermesini sağlar.
2.  **Projeyi Klonlar/Günceller:** Gerekli kodları GitHub'dan çeker.
3.  **Kütüphaneleri Yükler:** `requirements_enhanced.txt` dosyasındaki tüm bağımlılıkları kurar.
4.  **Python Path'i Ayarlar:** Proje kaynak kodunun import edilebilmesini sağlar.

In [ ]:
# 1. İnteraktif arayüzü etkinleştir (EN ÖNEMLİ ADIM)
from google.colab import output
output.enable_custom_widget_manager()

# 2. Gerekli diğer kütüphaneleri import et
import os
import sys
import subprocess

# 3. Ortam kurulum fonksiyonu
def setup_environment():
    """Clones or updates the repository and sets up the environment."""
    repo_url = "https://github.com/onndd/predictor_1.git"
    project_dir = "/content/predictor_1"

    if os.path.exists(project_dir):
        print("📁 Proje zaten mevcut. Güncelleniyor...")
        os.chdir(project_dir)
        # Pull işleminden önce olası yerel değişiklikleri sakla
        subprocess.run(["git", "stash"], check=True, capture_output=True)
        subprocess.run(["git", "pull", "origin", "main"], check=True, capture_output=True)
        # Saklanan değişiklikleri geri yüklemeye çalış (başarısız olabilir, sorun değil)
        subprocess.run(["git", "stash", "pop"], check=False, capture_output=True)
    else:
        print("📥 GitHub'dan proje klonlanıyor...")
        os.chdir("/content")
        subprocess.run(["git", "clone", repo_url], check=True, capture_output=True)
        os.chdir(project_dir)

    src_path = os.path.join(project_dir, "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"✅ {src_path} Python path'e eklendi")

    requirements_path = os.path.join(project_dir, 'requirements_enhanced.txt')
    if os.path.exists(requirements_path):
        print("📦 Gerekli kütüphaneler yükleniyor...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", requirements_path])
        print("✅ Kütüphaneler başarıyla yüklendi.")

# 4. Kurulumu çalıştır
try:
    setup_environment()
    print("\n🎉 Kurulum başarıyla tamamlandı! Sonraki hücreye geçebilirsiniz.")
except Exception as e:
    print(f"\n❌ Kurulum sırasında bir hata oluştu: {e}")

## 📊 2. MLflow Deney Takibini Başlatma (İsteğe Bağlı)

Eğitim sürecini ve metrikleri detaylı olarak izlemek isterseniz bu hücreyi çalıştırabilirsiniz. Sizden bir `ngrok` token isteyecektir.

In [ ]:
from getpass import getpass
from colab.health_check import ColabHealthCheck
import time

NGROK_AUTH_TOKEN = getpass('Lütfen ngrok authtokeninizi girin: ')

health_checker = ColabHealthCheck()
ngrok_ok, error_message = health_checker.check_ngrok_tunnel(NGROK_AUTH_TOKEN)

if ngrok_ok:
    print("✅ ngrok bağlantı testi başarılı.")
    get_ipython().system_raw('mlflow ui --host 0.0.0.0 &')
    time.sleep(5)
    print("\n✅ MLflow arayüzü için public URL oluşturuluyor...")
    print("👇 Aşağıdaki linke tıklayarak deneyleri izleyebilirsiniz:")
    try:
        from pyngrok import ngrok
        public_url = ngrok.connect(5000, "http").public_url
        print(f"--> {public_url}")
    except Exception as e:
        print(f"❌ ngrok tüneli oluşturulamadı. Hata: {e}")
else:
    print(f"\n❌ ngrok ön kontrolü başarısız oldu. Lütfen authtoken'ınızı ve internet bağlantınızı kontrol edin.")
    print(f"   Hata Detayı: {error_message}")

## 🚀 3. Modelleri Seç, Eğit ve İndir

Bu hücre, model seçimi yapmanızı, eğitimi başlatmanızı ve tamamlandığında eğitilmiş modelleri indirmenizi sağlar.

In [ ]:
import torch
import ipywidgets as widgets
from IPython.display import display, HTML, FileLink
from src.training.pipeline_manager import PipelineManager
from src.config.settings import get_all_models
import traceback

def run_training_interface():
    """
    Handles the entire process of creating and displaying the training UI,
    including model download links after training.
    """
    output_area = widgets.Output()
    download_links_area = widgets.VBox([]) # İndirme linkleri için ayrı bir alan

    def on_button_clicked(b, model_checkboxes, output_area):
        """
        This function is executed when the 'Start Training' button is clicked.
        It captures the selected models, initiates the training pipeline,
        and displays download links upon completion.
        """
        with output_area:
            output_area.clear_output()
            download_links_area.children = [] # Eski linkleri temizle

            selected_models = [cb.description for cb in model_checkboxes if cb.value]

            if not selected_models:
                print("⚠️ Lütfen en az bir model seçin!")
                return

            print(f"🚀 Seçilen modellerle eğitim başlatılıyor: {', '.join(selected_models)}")
            
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
            if device == 'cuda':
                print(f"✅ GPU bulundu: {torch.cuda.get_device_name(0)}")
            else:
                print("⚠️ GPU bulunamadı. Eğitim CPU ile devam edecek.")
            
            try:
                pipeline_manager = PipelineManager(models_to_train=selected_models, device=device)
                # Pipeline'dan model yollarını al
                trained_model_paths = pipeline_manager.run_pipeline()
                
                print("\n🎉 Eğitim tamamlandı! İndirme linkleri hazırlanıyor...")
                
                if trained_model_paths:
                    links = [FileLink(path) for path in trained_model_paths]
                    download_links_area.children = [widgets.HTML("<h3>⬇️ Eğitilmiş Modelleri İndir:</h3>")] + links
                else:
                    print("❌ Eğitilmiş model bulunamadı. Link oluşturulamadı.")

            except Exception as e:
                print(f"❌ Eğitim sırasında beklenmedik bir hata oluştu: {e}")
                traceback.print_exc()

    try:
        print("🔍 Eğitilebilir modeller alınıyor...")
        available_models = get_all_models()
        
        if not available_models:
            with output_area:
                print("❌ UYARI: `get_all_models()` fonksiyonu boş bir liste döndürdü. `config.yaml` dosyasında modellerin tanımlı olduğundan emin olun.")
            display(output_area)
            return
        
        print(f"✅ Başarıyla {len(available_models)} model bulundu.")

        print("🎨 Arayüz elemanları oluşturuluyor...")
        model_checkboxes = [widgets.Checkbox(value=False, description=model) for model in available_models]
        start_button = widgets.Button(description='Eğitimi Başlat', button_style='success', icon='rocket')
        
        start_button.on_click(lambda b: on_button_clicked(b, model_checkboxes, output_area))

        print("🖥️ Arayüz oluşturuluyor ve görüntüleniyor...")
        
        ui_container = widgets.VBox([
            widgets.HTML("<h3>Eğitmek istediğiniz modelleri seçin:</h3>"),
            *model_checkboxes,
            start_button,
            output_area,
            download_links_area # İndirme linkleri alanını ekle
        ])
        
        display(ui_container)
        
        with output_area:
            print("✅ Arayüz başarıyla görüntülendi. Lütfen model seçimi yapın.")

    except Exception as e:
        with output_area:
            output_area.clear_output()
            print(f"❌ HATA: Arayüz oluşturulurken kritik bir hata oluştu: {e}")
            traceback.print_exc()
        display(output_area)

# --- Arayüzü Başlat ---
run_training_interface()